In [55]:
import os
import sys
import json
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from matplotlib import pyplot as plt
import mne
from natsort import natsorted

import glob
import re
from itertools import groupby
from tqdm import tqdm
import pickle

mne.set_log_level("WARNING")
sys.path.append('/home/maxihuber/eeg-foundation')

tmp="""
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchaudio
import lightning as L

L.seed_everything(0, workers=True)
"""

## Pretrain: Prepare EyeNet Data

## Extract embeddings

In [12]:
class FinetuneDataset(Dataset):
    def __init__(self, task_name, task_index):
        self.task_name = task_name
        self.task_index = task_index

    def __getitem__(self, idx):
        task_dict = self.task_index[idx]

        task_data = df.empty
        
        for sub_task_path in task_dict["input"]:

            sub_task_data = df.read_pickle(sub_task_path)
            # concatenate along first axis with task_data

        # only take task_data[task_dict["start"]:task_dict["start"]+task_dict["length"]]

        # make torch.
            

        return None

    def __len__(self):
        return len(self.task_index)

In [11]:

class FinetuneModel(nn.Module):
    def __init__(self):
        super().__init__()

        # Define layers

    def forward(self, x):
        

    


def extract_embeddings(finetune_json):

    task_type = finetune_json["task_type"]
    
    with open(finetune_json["train"]["task_file"], 'r') as train_file:
        train_name = finetune_json["train"]["task_name"]
        train_index = json.load(train_file)
    with open(finetune_json["val"]["task_file"], 'r') as val_file:
        val_name = finetune_json["val"]["task_name"]
        val_index = json.load(val_file)
    with open(finetune_json["test"]["task_file"], 'r') as test_file:
        test_name = finetune_json["test"]["task_name"]
        test_index = json.load(test_file)

    # Initialize Datasets
    train_dataset = FinetuneDataset(task_name=train_name, task_index=train_index)
    val_dataset = FinetuneDataset(task_name=val_name, task_index=val_index)
    test_dataset = FinetuneDataset(task_name=test_name, task_index=test_index)

    # Initialize Dataloaders

    # Initialize network that extracts the encoder embeddings from the pre-trained model
    

EEGEyeNet_LR = {
    "train": {"task_name": "EEGEyeNet_LR_train",
              "task_file": "/itet-stor/maxihuber/deepeye_storage/eegeyenet_tasks/EEGEyeNet_LR_train.json"},
    "val": {"task_name": "EEGEyeNet_LR_val",
            "task_file": "/itet-stor/maxihuber/deepeye_storage/eegeyenet_tasks/EEGEyeNet_LR_val.json"},
    "test": {"task_name": "EEGEyeNet_LR_test",
             "task_file": "/itet-stor/maxihuber/deepeye_storage/eegeyenet_tasks/EEGEyeNet_LR_test.json"},
    "task_type": "Classification"
}

print(EEGEyeNet_LR)

{'train': {'task_name': 'EEGEyeNet_LR_train', 'task_file': '/itet-stor/maxihuber/deepeye_storage/eegeyenet_tasks/EEGEyeNet_LR_train.json'}, 'val': {'task_name': 'EEGEyeNet_LR_val', 'task_file': '/itet-stor/maxihuber/deepeye_storage/eegeyenet_tasks/EEGEyeNet_LR_val.json'}, 'test': {'task_name': 'EEGEyeNet_LR_test', 'task_file': '/itet-stor/maxihuber/deepeye_storage/eegeyenet_tasks/EEGEyeNet_LR_test.json'}, 'task_type': 'Classification'}


In [ ]:
df = pd.read_pickle('/itet-stor/maxihuber/deepeye_storage/eegeyenet_prepared/antisaccade_task_data-synchronised_min-BK2-oip_BK2_AS_EEG-segment-000000002-fixation.pickle')
print(df.columns.to_list())
for col in df.columns.to_list():
    print(f"{col}:", df.iloc[0][col])

## Train with Embeddings

### TUAB

In [17]:
train_path = '/home/maxihuber/eeg-foundation/development/index_files/tuab_train_labeled'
eval_path = '/home/maxihuber/eeg-foundation/development/index_files/tuab_eval_labeled'

full_tueg_path = '/itet-stor/maxihuber/deepeye_storage/index_files/full_tueg_index.json'

with open(train_path, 'r') as file:
    train_index = json.load(file)

with open(eval_path, 'r') as file:
    eval_index = json.load(file)

with open(full_tueg_path, 'r') as file:
    tueg_index = json.load(file)

tueg_to_ie = {ie["path"]: ie for ie in tueg_index}
path_prefix = '/itet-stor/maxihuber/deepeye_storage/foundation/tueg/edf'

print(len(train_index))
print(len(eval_index))

def add_metadata(tuab_index):
    tuab_index_new = []

    for i, [path, label] in enumerate(tuab_index):

        eeg_data = mne.io.read_raw_edf(
            path_prefix + path,
            preload=True,
        )

        sr = 1 / (eeg_data.times[1] - eeg_data.times[0])
        dur = len(eeg_data.times) / sr
        ie = {
            "path": path_prefix + path,
            "channels": eeg_data.info.ch_names,
            "sr": sr,
            "duration": dur,
            "label": label,
        }

        tuab_index_new.append(ie)
        print(f"i: {i}/{len(tuab_index)}, sr: {sr}, dur: {dur}")
        
    return tuab_index_new

train_index_new = add_metadata(train_index)
eval_index_new = add_metadata(eval_index) 

print(len(train_index_new))
print(len(eval_index_new))

tuab_train_index_path = '/itet-stor/maxihuber/deepeye_storage/index_files/tuab_train_index.json'
tuab_eval_index_path = '/itet-stor/maxihuber/deepeye_storage/index_files/tuab_test_index.json'

with open(tuab_train_index_path, 'w') as file:
    json.dump(train_index_new, file, indent=4)

with open(tuab_eval_index_path, 'w') as file:
    json.dump(eval_index_new, file, indent=4)

2717
276
i: 0/2717, sr: 250.0, dur: 1268.0
i: 1/2717, sr: 250.0, dur: 1317.0
i: 2/2717, sr: 250.0, dur: 1210.0
i: 3/2717, sr: 250.0, dur: 1219.0
i: 4/2717, sr: 250.0, dur: 1338.0
i: 5/2717, sr: 250.0, dur: 1325.0
i: 6/2717, sr: 250.0, dur: 1182.0
i: 7/2717, sr: 250.0, dur: 1206.0
i: 8/2717, sr: 250.0, dur: 1506.0
i: 9/2717, sr: 250.0, dur: 1326.0
i: 10/2717, sr: 250.0, dur: 1390.0
i: 11/2717, sr: 250.0, dur: 1477.0
i: 12/2717, sr: 250.0, dur: 1125.0
i: 13/2717, sr: 250.0, dur: 1443.0
i: 14/2717, sr: 250.0, dur: 1539.0
i: 15/2717, sr: 250.0, dur: 1205.0
i: 16/2717, sr: 250.0, dur: 1334.0
i: 17/2717, sr: 250.0, dur: 1203.0
i: 18/2717, sr: 256.0, dur: 1442.0
i: 19/2717, sr: 250.0, dur: 1202.0
i: 20/2717, sr: 250.0, dur: 1237.0
i: 21/2717, sr: 250.0, dur: 1187.0
i: 22/2717, sr: 250.0, dur: 1315.0
i: 23/2717, sr: 250.0, dur: 1191.0
i: 24/2717, sr: 250.0, dur: 1266.0
i: 25/2717, sr: 250.0, dur: 1409.0
i: 26/2717, sr: 250.0, dur: 1328.0
i: 27/2717, sr: 250.0, dur: 1455.0
i: 28/2717, sr: 250.0

In [26]:
print(len(os.listdir('/itet-stor/maxihuber/deepeye_storage/eegeyenet_prepared_concat')))

eyenet_index_path = '/itet-stor/maxihuber/deepeye_storage/index_files/full_eyenet_index.json'
with open(eyenet_index_path, 'r') as file:
    eyenet_index = json.load(file)

print(len(eyenet_index))

195
195


In [ ]:
checkpoint = torch.load('/itet-stor/maxihuber/net_scratch/checkpoints/978072/epoch=0-step=11861-val_loss=145.10.ckpt', map_location=torch.device('cpu'))

In [42]:
print(checkpoint.keys())
print(checkpoint['state_dict'])




dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers', 'datamodule_hparams_name', 'datamodule_hyper_parameters'])


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [3]:
import torch

def preprocess_checkpoint(checkpoint_path, new_checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
    
    # Transform the state_dict keys by stripping "net."
    state_dict = {key.replace('net.', ''): value for key, value in checkpoint['state_dict'].items()}
    checkpoint['state_dict'] = state_dict
    
    # Save the modified checkpoint
    torch.save(checkpoint, new_checkpoint_path)

# Define paths
original_checkpoint_path = '/itet-stor/maxihuber/net_scratch/checkpoints/978072/epoch=0-step=11861-val_loss=145.10.ckpt'
new_checkpoint_path = '/itet-stor/maxihuber/net_scratch/checkpoints/978072/epoch=0-step=11861-val_loss=145.10_modified.ckpt'

# Preprocess the checkpoint
preprocess_checkpoint(original_checkpoint_path, new_checkpoint_path)

In [9]:
from src.models.mae_rope_net import ModularMaskedAutoencoderViTRoPE
from src.models.mae_rope_module import MAEModuleRoPE

checkpoint_path = original_checkpoint_path
checkpoint = torch.load(checkpoint_path, map_location=torch.device("cpu"))

net = ModularMaskedAutoencoderViTRoPE("/home/maxihuber/eeg-foundation/src/data/components/channels_to_id.json")
state_dict = {
    key.replace("net.", ""): value
    for key, value in checkpoint["state_dict"].items()
}
keys1 = net.state_dict().keys()
net.load_state_dict(state_dict)
is_nan = torch.stack([torch.isnan(p).any() for p in net.parameters()]).any()
print(is_nan)
keys2 = list(net.state_dict().keys())[:5]

#print(keys1)
print("-" * 100)
print(keys2)

model = MAEModuleRoPE(net)
#keys3 = list(model.state_dict().keys())[:5]
#print(keys3)
keys4 = list(checkpoint["state_dict"].keys())[:5]
print(keys4)
model.load_state_dict(state_dict)
model = MAEModuleRoPE.load_from_checkpoint(checkpoint_path)

model.optimizer = model.configure_optimizers()
model.optimizer.load_state_dict(checkpoint["optimizer_states"][0])
model.scheduler.load_state_dict(checkpoint["scheduler_states"][0])

tensor(False)
----------------------------------------------------------------------------------------------------
['encoder.cls_token', 'encoder.patch_embed.proj.weight', 'encoder.patch_embed.proj.bias', 'encoder.mean_embed.weight', 'encoder.mean_embed.bias']
['net.encoder.cls_token', 'net.encoder.patch_embed.proj.weight', 'net.encoder.patch_embed.proj.bias', 'net.encoder.mean_embed.weight', 'net.encoder.mean_embed.bias']


RuntimeError: Error(s) in loading state_dict for MAEModuleRoPE:
	Missing key(s) in state_dict: "net.encoder.cls_token", "net.encoder.patch_embed.proj.weight", "net.encoder.patch_embed.proj.bias", "net.encoder.mean_embed.weight", "net.encoder.mean_embed.bias", "net.encoder.std_embed.weight", "net.encoder.std_embed.bias", "net.encoder.channel_encoding_map.None.param", "net.encoder.channel_encoding_map.20.param", "net.encoder.channel_encoding_map.21.param", "net.encoder.channel_encoding_map.22.param", "net.encoder.channel_encoding_map.23.param", "net.encoder.channel_encoding_map.24.param", "net.encoder.channel_encoding_map.25.param", "net.encoder.channel_encoding_map.26.param", "net.encoder.channel_encoding_map.27.param", "net.encoder.channel_encoding_map.28.param", "net.encoder.channel_encoding_map.29.param", "net.encoder.channel_encoding_map.30.param", "net.encoder.channel_encoding_map.31.param", "net.encoder.channel_encoding_map.32.param", "net.encoder.channel_encoding_map.33.param", "

In [10]:
import json

# Open and load the full index file
with open('/itet-stor/maxihuber/deepeye_storage/index_files/full_tueg_index.json', 'r') as file:
    tueg_index = json.load(file)

# Take every tenth element of this list
tueg_index_small = tueg_index[::10]  # Slicing with step 10

# Write the smaller index list to a new JSON file
with open('/itet-stor/maxihuber/deepeye_storage/index_files/10percent_tueg_index.json', 'w') as file:
    json.dump(tueg_index_small, file)


In [33]:
with open('/dev/shm/mae/data_index_full.json', 'r') as file:
    index = json.load(file)

for k, v in index.items():
    v["origin_path"] = os.path.basename(v["origin_path"])

assert False, "break before dumping"
with open('/dev/shm/mae/data_index_full.json', 'w') as file:
    json.dump(index, file)

In [32]:
index['0']

{'num_channels': 12,
 'channels': ['FP1',
  'F5',
  'AFZ',
  'F6',
  'CZ',
  'PZ',
  'P4',
  'P8',
  'O1',
  'OZ',
  'O2',
  'AVG_REF'],
 'paths': ['/dev/shm/mae/3_0/signal_3_0.npy',
  '/dev/shm/mae/3_0/signal_3_1.npy',
  '/dev/shm/mae/3_0/signal_3_2.npy',
  '/dev/shm/mae/3_0/signal_3_3.npy',
  '/dev/shm/mae/3_0/signal_3_4.npy',
  '/dev/shm/mae/3_0/signal_3_5.npy',
  '/dev/shm/mae/3_0/signal_3_6.npy',
  '/dev/shm/mae/3_0/signal_3_7.npy',
  '/dev/shm/mae/3_0/signal_3_8.npy',
  '/dev/shm/mae/3_0/signal_3_9.npy',
  '/dev/shm/mae/3_0/signal_3_10.npy',
  '/dev/shm/mae/3_0/signal_3_11.npy'],
 'sr': 512,
 'durs': [5.904296875,
  5.904296875,
  5.904296875,
  5.904296875,
  5.904296875,
  5.904296875,
  5.904296875,
  5.904296875,
  5.904296875,
  5.904296875,
  5.904296875,
  5.904296875],
 'ref': None,
 'Dataset': 'ERP_ERP_ANA',
 'SubjectID': '20e564dc-c201-11ee-a936-001e67445825',
 'origin_path': 'ERP_ERP_ANA_Subject4_20e564dc-c201-11ee-a936-001e67445825_session6_run3_trial23.pkl'}

In [29]:
with open('/home/maxihuber/eeg-foundation/src/data/components/mi_overall.json', 'r') as file:
    erp = json.load(file)

subset = 'train'
erp_subset = erp[subset]
paths = set()

for trial in erp_subset:
    paths.add(list(trial.keys())[0])

paths = natsorted(list(paths))

assert False, "break before dumping"
with open(f'/home/maxihuber/eeg-foundation/src/data/components/mi_{subset}_paths.json', 'w') as file:
    json.dump(paths, file, indent=4)

In [35]:
# Load information on what files to exclude
test_trial_paths = set()
with open('/home/maxihuber/eeg-foundation/src/data/components/mi_test_paths.json', 'r') as file:
    mi_test_paths = json.load(file)
    test_trial_paths.update(mi_test_paths)
with open('/home/maxihuber/eeg-foundation/src/data/components/erp_test_paths.json', 'r') as file:
    erp_test_paths = json.load(file)
    test_trial_paths.update(erp_test_paths)
print(len(test_trial_paths))

55986


In [48]:
with open('/dev/shm/mae/data_index_0.txt', 'r') as file:
    index = json.load(file)

print(index['0'])

for trial_idx, trial_info in index.items():
    new_paths = [path.replace('/scratch/mae', '/dev/shm/mae') for path in trial_info["paths"]]
    trial_info["paths"] = new_paths

assert False
with open('/dev/shm/mae/data_index_0.txt', 'w') as file:
    json.dump(index, file)

{'num_channels': 27, 'channels': ['EEG FP1-REF', 'EEG FP2-REF', 'EEG F3-REF', 'EEG F4-REF', 'EEG C3-REF', 'EEG C4-REF', 'EEG P3-REF', 'EEG P4-REF', 'EEG O1-REF', 'EEG O2-REF', 'EEG F7-REF', 'EEG F8-REF', 'EEG T3-REF', 'EEG T4-REF', 'EEG T5-REF', 'EEG T6-REF', 'EEG A1-REF', 'EEG A2-REF', 'EEG FZ-REF', 'EEG CZ-REF', 'EEG PZ-REF', 'EEG ROC-REF', 'EEG LOC-REF', 'EEG EKG1-REF', 'EEG T1-REF', 'EEG T2-REF', 'AVG_REF'], 'paths': ['/dev/shm/mae/0_0/signal_0_0.npy', '/dev/shm/mae/0_0/signal_0_1.npy', '/dev/shm/mae/0_0/signal_0_2.npy', '/dev/shm/mae/0_0/signal_0_3.npy', '/dev/shm/mae/0_0/signal_0_4.npy', '/dev/shm/mae/0_0/signal_0_5.npy', '/dev/shm/mae/0_0/signal_0_6.npy', '/dev/shm/mae/0_0/signal_0_7.npy', '/dev/shm/mae/0_0/signal_0_8.npy', '/dev/shm/mae/0_0/signal_0_9.npy', '/dev/shm/mae/0_0/signal_0_10.npy', '/dev/shm/mae/0_0/signal_0_11.npy', '/dev/shm/mae/0_0/signal_0_12.npy', '/dev/shm/mae/0_0/signal_0_13.npy', '/dev/shm/mae/0_0/signal_0_14.npy', '/dev/shm/mae/0_0/signal_0_15.npy', '/dev/sh

AssertionError: 

In [49]:
with open('/itet-stor/maxihuber/deepeye_storage/index_files/full_pkl_index.json', 'r') as file:
    pkl_index = json.load(file)

dur = 0
for ie in pkl_index:
    dur += ie["duration"] * len(ie["channels"])

print(dur)

76891825.02574204


In [56]:
class TaskCreator:
    def __init__(self, prepared_data_directory='./', 
                       save_task_directory="./", 
                       load_file_pattern='*', 
                       task_name='all.json', 
                       task_type='Classification',
                       verbose=False):

        self.prepared_data_directory = prepared_data_directory
        self.save_task_directory = save_task_directory
        #self.load_file_pattern = re.compile(load_file_pattern)
        self.task_name = task_name
        self.task_type = task_type
        self.verbose = verbose
        
        self.start_time = None
        self.length_time = None
        self.start_channel = 1
        self.end_channel = 129
        self.on_blocks = None
        self.off_blocks = None
        self.filters = []
        self.ignore_events = []
        self.labels = []

        print("Task creator is initialized with: ")
        print("Directory to load prepared data: " + self.prepared_data_directory)
        print("Directory to save tasks: " + self.save_task_directory)
        print("Task name: " + self.task_name)
        print("Looking for file that match: " + load_file_pattern)


    def extract_data_at_events(self, extract_pattern, name_start_time, start_time, name_length_time, length_time):
        self.extract_pattern = extract_pattern
        self.start_time = start_time
        self.length_time = length_time

        print("Task creator is instructed to look for units that match structure: " + str(self.extract_pattern))
        print("Time dimension -- Cut start info: " + name_start_time)
        print("Time dimension -- Cut length info: " + name_length_time)

    def extract_data_continuously(self, name_length_time, length_time):
        self.length_time = length_time

        print("Task creator is instructed to extract data continuosly! ")
        print("Time dimension -- Cut length info: " + name_length_time)

    def ignoreEvent(self, name, f):
        self.ignore_events.append((name, f))
        print('Task creator is instructed to ignore the event: ' + name)

    def blocks(self, on_blocks, off_blocks):
        self.on_blocks = on_blocks
        self.off_blocks = off_blocks
        print("Blocks to be used are: " + str(on_blocks))
        print("Blocks to be ignored are: " + str(off_blocks))

    def addFilter(self, name, f):
        self.filters.append((name, f))
        print('Task creator is instructed to use filter: ' + name)

    def addLabel(self, name, f):
        self.labels.append((name, f))
        print('Task creator is instructed to use label: ' + name)

    def process_stream(self, stream):
        # load the pickle files
        if self.verbose: print(f"Stream is composed of {len(stream)} eye movement events.")
        stream_df, indices_in_order = self._read_stream(stream)
        if not indices_in_order:
            raise Exception(f"Indices are not in order in the sample: {stream_df}")
        events_df = self._get_event_info(stream_df)
        if self.verbose: print(events_df["type"])
        events_df = self._ignore_events(events_df)
        select = self._filter_blocks(events_df)
        select &= self._filter_pattern(events_df)
        select &= self._filter_events(events_df, stream_df)
        events_df = self._compute_labels(events_df)
        if hasattr(self, 'extract_pattern'):
            samples = self._extract_samples(stream_df, events_df, select)
        else:
            samples = self._extract_samples_continuously(stream_df, events_df, select)
        if self.verbose: print(f"Finished processing stream, and got {len(samples)} samples")
        return samples

    def process_subject(self, dataset_group, dataset, subject):
        print("Starting creating task...")
        all_units = sorted(os.listdir(self.prepared_data_directory))
        print("Length of all units: ", len(all_units))
        all_units = [unit for unit in all_units if self.load_file_pattern.match(unit) and f"{dataset_group}-{dataset}-{subject}" in unit]   
        print("Length of all units after pattern: ", len(all_units))
        all_streams = self._create_streams(all_units) # TODO
        print("Length of all streams: ", len(all_streams))
        all_samples = []
        for stream in all_streams:
            samples = self.process_stream(stream)
            all_samples.extend(samples)
        return all_samples

    def run_sequential(self, user_ids=None, split="", stream_pattern=None):
        print("Starting creating task...")
        all_units = sorted(os.listdir(self.prepared_data_directory))
        print("Length of all units: ", len(all_units))
        all_units = [unit for unit in all_units if self.load_file_pattern.match(unit)]
        print("Length of all units after pattern: ", len(all_units))
        all_streams = self._create_streams(all_units, stream_pattern)
        print("Length of all streams: ", len(all_streams))
        chosen_samples = []
        progress = tqdm(all_streams)
        for stream, user_id in progress:
            if user_ids is not None and user_id.lower() not in user_ids:
                continue
            samples = self.process_stream(stream)
            chosen_samples.extend(samples)
        # Save as JSON the chosen samples
        print(f"Finished creating task, and got {len(chosen_samples)} samples")
        if self.verbose: print(chosen_samples[:4])
        task = {self.task_name + "_" + split: chosen_samples, "task_type": self.task_type}
        json.dump(task, open(os.path.join(self.save_task_directory, self.task_name + "_" + split + ".json"), 'w'))

    def _get_event_info(self, sample_df):
        events = sample_df.copy() # sample_df[sample_df['type'] != sample_df['type'].shift()]
        events = events.reset_index(drop=False)
        events = events.apply(pd.Series.explode)
        events = events[events['type'] != events['type'].shift()]
        return events

    def get_relative_path(self, full_path, levels_to_remove=2):
        # Split the path into its components
        path_parts = full_path.split(os.sep)
        
        # Join the path parts starting from the specified level
        relative_path = os.sep.join(path_parts[levels_to_remove:])
        
        return relative_path

    def _read_stream(self, stream):
        sample_dfs = []
        for event in stream:
            next_df = pd.read_pickle(os.path.join(self.prepared_data_directory, event))
            next_df["picklename"] = str(self.get_relative_path(os.path.join(self.prepared_data_directory, event)))
            # if self.verbose: print("Reading", str(os.path.join(self.prepared_data_directory, event)))
            sample_dfs.append(next_df)
        sample_df = pd.concat(sample_dfs, axis=0)
        sample_df = self.fix_types(sample_df)
        indices_in_order = (sample_df.index.to_series().diff().fillna(1) == 1).all()
        return sample_df, indices_in_order

    def _create_streams(self, all_units, stream_pattern=None):
        # Regex pattern to extract the session part of the filename
        pattern = re.compile(r'(.+segment-)')
        # Grouping filenames using groupby
        grouped_units = [list(group) for key, group in groupby(all_units, key=lambda x: pattern.match(x).group(1))]
        if stream_pattern is not None:
            streams = []
            # If stream pattern is not None, then we need to find the stream that matches the pattern
            for group in grouped_units:
                for i in range(len(group) - len(stream_pattern)):
                    valid_stream = True
                    for j in range(len(stream_pattern)):
                        valid_stream = valid_stream and (stream_pattern[j] in group[i + j])
                    stream = [group[i + j] for j in range(len(stream_pattern))]
                    if valid_stream:
                        streams.append(stream)
        else:
            streams = grouped_units

        streams_with_user_id = []
        for stream in streams:
            user_ids = {filename.split('-')[2] for filename in stream}
            if len(user_ids) != 1:
                raise ValueError(f"Stream contains multiple user IDs: {user_ids}")
            user_id = user_ids.pop()  # Get the single user ID
            streams_with_user_id.append((stream, user_id))
        return streams_with_user_id

    def _filter_blocks(self, events):
        select = events['type'].apply(lambda x: True)
        if self.on_blocks is None:
            return select
        if self.verbose: print("Filtering the blocks: ")
        select = events["block"].isin(self.on_blocks)
        if self.verbose: print(list(zip(events["type"].tolist(), events["latency"].tolist(), select))[:100])
        return select

    def _ignore_events(self, events):
        if self.verbose: print("Ignoring the events:")
        ignore = events['type'].apply(lambda x: False)
        for name, f in self.ignore_events:
            if self.verbose: print("Applying: " + name)
            ignore |= f(events)
            if self.verbose: print(list(zip(events["type"].tolist(), events["latency"].tolist(), ignore))[:100])
        select = ignore.apply(lambda x: not x)
        return events.loc[select]

    def _filter_pattern(self, events):
        select = events['type'].apply(lambda x: True)
        if not hasattr(self, 'extract_pattern'):
            return select
        if self.verbose: print(f"Checking the pattern: {self.extract_pattern}")
        for i, event in enumerate(self.extract_pattern):
            select &= events['type'].shift(-i).isin(event)
        if self.verbose: print(list(zip(events["type"].tolist(), events["latency"].tolist(), select))[:100])
        return select

    def _filter_events(self, events, stream=None):
        if self.verbose: print("Filtering the events")
        select = events['type'].apply(lambda x: True)
        for name, f in self.filters:
            if self.verbose: print("Applying filter: " + name)
            select &= f(events, stream)
            if self.verbose: print(list(zip(events["type"].tolist(), events["latency"].tolist(), select))[:100])
        return select
    
    def _compute_labels(self, events):
        if self.verbose: print("Computing the label.")
        for name, f in self.labels:
            if self.verbose: print("Appending the next label: " + name)
            events[f"label_{name}"] = f(events)
            if self.verbose: print(events[f"label_{name}"])
        return events

    def _extract_samples_continuously(self, stream, events, select):
        if self.verbose: print("Extracting data continuously")
        all_samples = []
        # extract the useful data
        if self.verbose: print(stream)
        
        events['group'] = (select != select.shift()).cumsum()
        filtered_groups = events.loc[select].groupby('group')
        start = []
        for _, group in filtered_groups: # For each stream
            # rint(group)
            start.extend(np.arange(group.iloc[0]['latency'], group.iloc[-1]['endtime'] - 500 + 1, 500))

        length = 500
        if self.verbose: print(start)
        if self.verbose: print(length)
        for s in start:
            if s + length <= stream.index[-1]:
                window = stream.loc[int(s):int(s + length)]
                sample = {"input": window["picklename"].unique().tolist(), "output": None, "start": int(s), "length": int(length)}
                all_samples.append(sample)
            else:
                # Possible to be "padding" able. (in case we need later)
                if self.verbose: print("Stream is not enough long to get the required length for the sample, skipping.")
        return all_samples

    def _extract_samples(self, stream, events, select):
        if self.verbose: print("Extracting data from the interested events: ")
        all_samples = []
        # extract the useful data
        if self.verbose: print(stream)

        start = self.start_time(events).loc[select]
        length = events['type'].apply(lambda x: self.length_time).loc[select]
        labels = events.filter(like='label_').loc[select]
        if self.verbose: print(start)
        if self.verbose: print(length)
        if self.verbose: print(labels)
        for s, l, (label_idx, label) in zip(start, length, labels.iterrows()):
            if s + l <= stream.index[-1]:
                window = stream.loc[int(s):int(s + l)]
                sample = {"input": window["picklename"].unique().tolist(), "output": label.to_dict(), "start": int(s), "length": int(l)}
                all_samples.append(sample)
            else:
                # Possible to be "padding" able. (in case we need later)
                if self.verbose: print("Stream is not enough long to get the required length for the sample, skipping.")
        return all_samples

    def fix_types(self, sample_df):
        sample_df['sac_endpos_x'] = pd.to_numeric(sample_df['sac_endpos_x'], errors='coerce')
        sample_df['sac_startpos_x'] = pd.to_numeric(sample_df['sac_startpos_x'], errors='coerce')
        sample_df['sac_endpos_y'] = pd.to_numeric(sample_df['sac_endpos_y'], errors='coerce')
        sample_df['sac_startpos_y'] = pd.to_numeric(sample_df['sac_startpos_y'], errors='coerce')
        sample_df['fix_avgpos_x'] = pd.to_numeric(sample_df['fix_avgpos_x'], errors='coerce')
        sample_df['fix_avgpos_y'] = pd.to_numeric(sample_df['fix_avgpos_y'], errors='coerce')
        return sample_df

task_creator = TaskCreator(prepared_data_directory='/itet-stor/maxihuber/deepeye_storage/eegeyenet_prepared')

print("Starting creating task...")
all_units = sorted(os.listdir(task_creator.prepared_data_directory))
print("Length of all units: ", len(all_units))
all_units = [unit for unit in all_units if unit != 'progress_log']
print("Length of all units: ", len(all_units))

# Regex pattern to extract the session part of the filename
pattern = re.compile(r'(.+segment-)')

# Grouping filenames using groupby
grouped_units = [list(group) for key, group in groupby(all_units, key=lambda x: pattern.match(x).group(1))]
streams = grouped_units

streams_with_user_id = []
for stream in streams:
    user_ids = {filename.split('-')[2] for filename in stream}
    if len(user_ids) != 1:
        raise ValueError(f"Stream contains multiple user IDs: {user_ids}")
    user_id = user_ids.pop()  # Get the single user ID
    streams_with_user_id.append((stream, user_id))

user_ids = [user_id for stream, user_id in streams_with_user_id]
print("# IDs:", len(streams_with_user_id))

Task creator is initialized with: 
Directory to load prepared data: /itet-stor/maxihuber/deepeye_storage/eegeyenet_prepared
Directory to save tasks: ./
Task name: all.json
Looking for file that match: *
Starting creating task...
Length of all units:  940553
Length of all units:  940552
# IDs: 593


In [59]:
for i, (stream, user) in enumerate(streams_with_user_id):
    if user == 'BE5':
        print(i)


195


In [58]:
with open('/itet-stor/maxihuber/deepeye_storage/index_files/full_eyenet_index.json', 'r') as file:
    eye1 = json.load(file)

with open('/itet-stor/maxihuber/deepeye_storage/index_files/full_eyenet_index2.json', 'r') as file:
    eye2 = json.load(file)

with open('/itet-stor/maxihuber/deepeye_storage/index_files/full_eyenet_index3.json', 'r') as file:
    eye3 = json.load(file)

with open('/itet-stor/maxihuber/deepeye_storage/index_files/full_eyenet_index4.json', 'r') as file:
    eye4 = json.load(file)

with open('/itet-stor/maxihuber/deepeye_storage/index_files/full_eyenet_index5.json', 'r') as file:
    eye5 = json.load(file)

with open('/itet-stor/maxihuber/deepeye_storage/index_files/full_eyenet_index6.json', 'r') as file:
    eye6 = json.load(file)

full_eye = eye1 + eye2 + eye3 + eye4 + eye5 + eye6
full_eye_ids = [ie["SubjectID"] for ie in full_eye]

def get_eye_dur(eye_index):
    dur = 0
    for ie in eye_index:
        dur += ie["duration"] * len(ie["channels"])
    return dur

print(len(eye1) + len(eye2) + len(eye3) + len(eye4) + len(eye5) + len(eye6))
print(get_eye_dur(eye1) + get_eye_dur(eye2) + get_eye_dur(eye3) + get_eye_dur(eye4) + get_eye_dur(eye5) + get_eye_dur(eye6))

print(len(full_eye_ids))
print(set(user_ids) - set(full_eye_ids))

592
44330484.15599998
592
{'BE5'}
